In [1]:
using Rx

In [2]:
source = file("data")

Rx.SyncFileObservable("data")

In [4]:
subscribe!(source, LambdaActor{String}(
    on_next = (d) -> println(d)
));

H
e
l
l
o
,
w
o
r
l
d
!


In [5]:
subscribe!(source |> uppercase(), LambdaActor{String}(
    on_next = (d) -> println(d)
));

H
E
L
L
O
,
W
O
R
L
D
!


In [7]:
subscribe!(source |> uppercase() |> reduce(String, *) |> map(String, reverse), LambdaActor{String}(
    on_next = (d) -> println(d)
));

HELLO,WORLD!


In [8]:
subscribe!(source |> lowercase() |> scan(String, (s, c) -> c * s), LambdaActor{String}(
    on_next = (d) -> println(d)
));

h
he
hel
hell
hello
hello,
hello,w
hello,wo
hello,wor
hello,worl
hello,world
hello,world!
